# C3 — Atualização de Pose (posição e orientação) em Servo Visual (Difícil)

Estime o delta de pose (posição e orientação) — [rot_z, tx, ty] — a partir de correspondências esparsas de features. Retorne estimativas robustas com ~20% de outliers.

### Contexto
Servo visual alinha a pose (posição e orientação) da câmera do robô a uma vista desejada rastreando correspondências de features entre quadros consecutivos. Os pares de pontos $(X1, X2)$ incluem inliers que obedecem a um pequeno delta de pose em SE(2) e outliers de mismatches/oclusões. Sintetizamos dados via [`common.data_utils.visual_servo_dataset`](common/data_utils.py): cada amostra tem até 20% de outliers mais ruído de sensor, e deltas verdadeiros $[\Delta\theta_z, \Delta t_x, \Delta t_y]$ dentro de ±0.01–0.02 radianos/metros.

### Por que quântico?
Estimação robusta de pose com outliers pode ser formulada como busca combinatória por consenso (variantes de RANSAC). Sub‑rotinas quânticas (por exemplo, amplificação de amplitude para contagem de inliers, QAOA para pontuar subconjuntos) podem acelerar a avaliação de hipóteses ou explorar conjuntos de consenso mais amplos em comparação a heurísticas puramente clássicas.

## Especificação de Entrada e Saída
**Entrada:** `X1 (N×M×2)`, `X2 (N×M×2)`  
**Saída:** `dpose (N×3)` com colunas `[rot, tx, ty]`  
**Tipo de retorno:** array float (`np.float64` recomendado)

Orientações de runtime/recursos: qubits ≤ 12, passos do otimizador ≤ 250; mantenha o tempo por amostra ≤ ~0,2 s.

### Dicas de solução
- Objetivo robusto: minimizar resíduos de Tukey/Huber entre $X1$ transformado e $X2$ ou maximizar o número de inliers sob um limiar de consenso.
- Ângulo quântico: pontuar subconjuntos candidatos via QAOA/Ising (selecionar inliers) ou usar estimação de amplitude para avaliar o suporte da hipótese de forma eficiente.
- Fluxo prático: (1) gerar hipóteses de pose (posição e orientação) — por exemplo, solucionadores mínimos com 2–3 correspondências —, (2) avaliar/impulsionar o consenso com método quântico, (3) refinar a melhor pose com mínimos quadrados ponderados.

_Orientação de baseline:_ [`common.baselines.baseline_visual_servo`](common/baselines.py) obtém medianas de erro ≈ [0.006 rad, 0.012 m, 0.012 m] em seeds variadas; testes públicos exigem medianas de tradução ≤ 0.02.

## Baseline (referência)

In [ ]:
X1, X2, y = du.visual_servo_dataset(n=64, m=24, seed=0, outlier_rate=0.2)
pred = bl.baseline_visual_servo(X1, X2)
err = np.median(np.abs(pred - y), axis=0)
print("Mediana do |erro| do baseline [rot, tx, ty]:", err)

## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(X1, X2):
    # Substitua por estimador robusto com reforço quântico (ex.: pontuação RANSAC quântica).
    return bl.baseline_visual_servo(X1, X2)

## Testes públicos (rápidos)

In [ ]:
X1, X2, y = du.visual_servo_dataset(n=64, m=24, seed=1, outlier_rate=0.2)
pred = solve(X1, X2)
med = np.median(np.abs(pred - y), axis=0)
print("Mediana pública |erro|:", med)
assert pred.shape == y.shape
assert med[1] <= 0.02 and med[2] <= 0.02, "Erro mediano de tradução muito alto"
print("OK")

> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.